<p align="center">
<img src="https://github.com/datacamp/data-analysis-in-sql-live-session/blob/master/assets/datacamp.svg?raw=True" alt = "DataCamp icon" width="50%">
</p>
<br><br>

## **Data Analysis with SQL**

In this webinar, you'll learn how to write advanced queries to calculate core business metrics and KPIs. You'll be able to:

* Use Common Table Expressions to temporarily store a query's results
* Fetch values from different rows using window functinos
* Use self-joins to peak into the future

## **The Dataset**


We'll use two tables. The first, `user_sessions`, stores data user session data on a social media website. The table's schema is as follows:

- `session_date`: The date on which the user accessed the site
- `user_id`: The user's unique identifier
- `time_spent_in_mins`: How much time the user spent on the site

The second, `user_data`, stores the users' metadata. The table's schema is as follows:

- `user_id`: The user's unique identifier
- `country`: The user's country
- `age`: The user's age


## **Setting up PostgreSQL**

In [ ]:
#@title **This block of code will install PosgreSQL**
%%capture
!wget -qO- https://www.postgresql.org/media/keys/ACCC4CF8.asc | apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ bionic-pgdg main" >/etc/apt/sources.list.d/pgdg.list
!apt -qq update
!apt -yq install postgresql-12 postgresql-client-12
!service postgresql start
# make calling psql shorter
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"  
!psql postgres -c "CREATE DATABASE root"  # now just !psql -c "..."
# load SQL extensions
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

In [ ]:
#@title **This will download your data to local environment**
!wget -q https://github.com/datacamp/data-analysis-in-sql-live-training/raw/master/data/user_data.csv
!wget -q https://github.com/datacamp/data-analysis-in-sql-live-training/raw/master/data/user_metadata.csv

In [ ]:
#@title **This will create your table**
%%sql
-- Make sure to amend you table name, column names and types
DROP TABLE IF EXISTS user_sessions;
CREATE TABLE user_sessions(
 session_date date,
 user_id int,
 time_spent_in_mins int
);

COPY user_sessions
-- Make sure to point to correct file and delimiter 
FROM '/content/user_data.csv' DELIMITER ',' CSV HEADER;

DROP TABLE IF EXISTS user_data;
CREATE TABLE user_data(
 user_id int,
 country char(3),
 age int
);

COPY user_data
-- Make sure to point to correct file and delimiter 
FROM '/content/user_metadata.csv' DELIMITER ',' CSV HEADER;

 * postgresql+psycopg2://@/postgres


""


Let's start by exploring the tables.

In [ ]:
%%sql

-- SELECT first 5 rows from user_sessions
SELECT *
FROM user_sessions
LIMIT 5;

 * postgresql+psycopg2://@/postgres


,session_date,user_id,time_spent_in_mins
0,2020-01-01,1,127
1,2020-01-01,2,147
2,2020-01-01,5,106
3,2020-01-01,6,179
4,2020-01-01,7,143


In [ ]:
%%sql

-- SELECT first 5 rows from user_data

SELECT *
FROM user_data
LIMIT 5;

 * postgresql+psycopg2://@/postgres


,user_id,country,age
0,0,GER,23
1,1,IND,23
2,2,IND,43
3,3,GER,22
4,4,JPN,47


In [ ]:
%%sql

-- Join the two tables together
SELECT 
  *
FROM user_sessions
JOIN user_data ON user_sessions.user_id = user_data.user_id
LIMIT 5

 * postgresql+psycopg2://@/postgres


,session_date,user_id,time_spent_in_mins,user_id,country,age
0,2020-01-01,1,127,1,IND,23
1,2020-01-01,2,147,2,IND,43
2,2020-01-01,5,106,5,GER,52
3,2020-01-01,6,179,6,GER,46
4,2020-01-01,7,143,7,GBR,59


## **Data overview**

Since you have user demographics, you can start by exploring some basic metrics, like:

- Average age per country
- User count by country

In [ ]:
%%sql

-- Get the average age per country
SELECT 
    country,
    ROUND(AVG(age), 2) AS avg_age
FROM user_data
GROUP BY country;


 * postgresql+psycopg2://@/postgres


,country,avg_age
0,JPN,41.78
1,IND,42.27
2,FRA,41.38
3,RUS,40.85
4,CHN,40.25
5,GBR,42.12
6,GER,41.49
7,USA,41.30


In [ ]:
%%sql

-- Get the user count by country 
SELECT 
    country,
    COUNT(DISTINCT user_id) AS users
FROM user_data
GROUP BY country;

 * postgresql+psycopg2://@/postgres


,country,users
0,CHN,245
1,FRA,227
2,GBR,266
3,GER,247
4,IND,289
5,JPN,276
6,RUS,248
7,USA,263


## **Active users**

The active users KPI counts the active users of a company's app over a certain time period:
- by day (daily active users, or DAU)
- by month (monthly active users, or MAU)

For example, Facebook had 1.76B DAU and 2.6 MAU in March.

Stickiness (DAU / MAU) measures how often users engage with an app on average. Facebook's stickiness for March was `1.76B / 2.6B ~= 0.677`, meaning that, on average, users used Facebook for `67.7% x 30 days ~= 20` days each month.

To get the daily active users, we need to count the number of unique `user_id`s for each `session_date`

In [ ]:
%%sql

-- Calculate the Daily Active Users (DAU)
SELECT 
  session_date,
  COUNT(DISTINCT user_id) AS users
FROM user_sessions
GROUP BY session_date
ORDER BY session_date ASC; 


 * postgresql+psycopg2://@/postgres


,session_date,users
0,2020-01-01,777
1,2020-01-02,755
2,2020-01-03,786
3,2020-01-04,760
4,2020-01-05,749
...,...,...
147,2020-05-27,1034
148,2020-05-28,1066
149,2020-05-29,1023
150,2020-05-30,1050


This is what the results would look like when visualized:

![Facespace DAU](https://github.com/datacamp/data-analysis-in-sql-live-training/raw/master/assets/facespace_dau.png)

## **Monthly active users**

Usually, reports include MAU, not DAU. How do you convert the session dates to months?

**Enter `DATE_TRUNC`**

`DATE_TRUNC(date_part, date) → DATE`: Truncates `date` to the nearest `date_part`.

**Examples**
- `DATE_TRUNC('week', '2018-06-12') :: DATE` → `'2018-06-11'`
- `DATE_TRUNC('month', '2018-06-12') :: DATE` → `'2018-06-01'`
- `DATE_TRUNC('quarter', '2018-06-12') :: DATE` → `'2018-04-01'`
- `DATE_TRUNC('year', '2018-06-12') :: DATE` → `'2018-01-01'`

**Note**: `:: DATE` is just to remove the hours, minutes, and seconds.

In [ ]:
%%sql

-- Calculate the Monthly Active Users (DAU)
SELECT DISTINCT
   DATE_TRUNC('month', session_date) :: DATE AS session_month,
   COUNT(DISTINCT user_id) AS users
FROM user_sessions
GROUP BY session_month 
ORDER BY session_month ASC;

 * postgresql+psycopg2://@/postgres


,session_month,users
0,2020-01-01,1472
1,2020-02-01,1621
2,2020-03-01,1723
3,2020-04-01,1879
4,2020-05-01,1995


This is what the results would look like when visualized:

![Facebook MAU](https://github.com/datacamp/data-analysis-in-sql-live-training/raw/master/assets/facespace_mau.png)

## **Q&A**

## **Registration dates**

Let's define the user's registration date as the date of that user's first session.

So, each user's registration date is the minimum session date for that user in the `user_sessions` table.

We'll use these results later on to calculate the growth in registrations.

In [ ]:
%%sql

-- Get each user's registration date
SELECT 
  user_id,
  MIN(session_date) AS registration_date
FROM user_sessions
GROUP BY user_id
ORDER BY registration_date ASC, user_id ASC;


 * postgresql+psycopg2://@/postgres


,user_id,registration_date
0,1,2020-01-01
1,2,2020-01-01
2,5,2020-01-01
3,6,2020-01-01
4,7,2020-01-01
...,...,...
2056,1953,2020-05-11
2057,2051,2020-05-11
2058,1969,2020-05-13
2059,2040,2020-05-15


## **Registrations and Common Table Expressions (CTEs)**

Now that you have each user's registration date, you'll want to store the results somehow to use them in a different query. How do you do that?

**Enter Common Table Expressions (CTEs)**

```sql
WITH cte_name AS (
  ...
)

SELECT *
FROM cte_name;
```

A CTE stores the results of a query temporarily in the specificed `cte_name` so it can be used in the outer query later on.

Once you store the results of the previous query in a CTE, you can `DATE_TRUNC()` the registration dates and count the unique `user_id`s in each registration month.

In [ ]:
%%sql

-- Store each user's registration date in the regs CTE
-- Calculate the number of registrations per month

WITH regs AS (
    -- where we write our query
  SELECT 
    user_id,
    MIN(session_date) AS registration_date
  FROM user_sessions
  GROUP BY user_id
)

SELECT 
  DATE_TRUNC('month', registration_date) :: DATE AS registration_month,
  COUNT(DISTINCT user_id) AS users
FROM regs
GROUP BY registration_month
ORDER BY registration_month ASC;

 * postgresql+psycopg2://@/postgres


,registration_month,users
0,2020-01-01,1472
1,2020-02-01,166
2,2020-03-01,147
3,2020-04-01,152
4,2020-05-01,124


## **Growth and window functions**

You now have each month's registrations. How do you calculate growth?

`Growth = (Current month - previous month) / previous month`

For example, if you had 122 registrations last month, and you have 156 registrations this month, your registrations grew by `(156 - 122) / 122 ~= 28%` this month.

So you need both the previous and the current months' registrations in the same row. How do you do that?

**Window functions**

A window function performs some operation across a set of table rows that are somehow related to the current row.

- `LAG(column_a, 1) OVER (ORDER BY column_b ASC)` Gets the previous row's value in `column_a` if you sort by `column_b`.

In [ ]:
%%sql
-- Fetch the previous and current months' MAUs

WITH regs AS (
SELECT 
  user_id,
  MIN(session_date) AS registration_date
FROM user_sessions
GROUP BY user_id),

    monthly_regs AS (
    SELECT
      DATE_TRUNC('month', registration_date) :: DATE AS registration_month,
      COUNT(DISTINCT user_id) AS users
    FROM regs
    GROUP BY registration_month
     ),

  prev_regs AS (  
  SELECT 
    registration_month,
    users,
    LAG(users, 1) OVER (ORDER BY registration_month ASC) AS prev_reg
  FROM monthly_regs
  )

  SELECT 
    registration_month,
    users,
    COALESCE(prev_reg, 1) AS prev_reg
  FROM prev_regs
  ORDER BY registration_month ASC;

 * postgresql+psycopg2://@/postgres


,registration_month,users,prev_reg
0,2020-01-01,1472,1
1,2020-02-01,166,1472
2,2020-03-01,147,166
3,2020-04-01,152,147
4,2020-05-01,124,152


Store the results in a CTE and apply the formula to get the monthly registrations growth rates. You can use `COALESCE(..., 1)` to convert any `NULL` values to 1.

In [ ]:
%%sql

-- Calculate the monthly growth in registrations
WITH regs AS (
SELECT 
  user_id,
  MIN(session_date) AS registration_date
FROM user_sessions
GROUP BY user_id),

    monthly_regs AS (
    SELECT
      DATE_TRUNC('month', registration_date) :: DATE AS registration_month,
      COUNT(DISTINCT user_id) AS users
    FROM regs
    GROUP BY registration_month
     ),

  prev_regs AS (  
  SELECT 
    registration_month,
    users,
    LAG(users, 1) OVER (ORDER BY registration_month ASC) AS prev_reg
  FROM monthly_regs
  )

  SELECT
    registration_month,
    ROUND(
    (users - COALESCE(prev_reg,1)) :: NUMERIC / COALESCE(prev_reg,1),3) AS growth_rate
  FROM prev_regs
  ORDER BY registration_month ASC;
  

 * postgresql+psycopg2://@/postgres


,registration_month,growth_rate
0,2020-01-01,1471.000
1,2020-02-01,-0.887
2,2020-03-01,-0.114
3,2020-04-01,0.034
4,2020-05-01,-0.184


## **Q&A**

## **Retained and resurrected users**

Users can be split into four groups:
- New/registered users are ones that just signed up for your platform
- Retained users used to use your app, and still do, too.
- Churned users used to use your app, and no longer do.
- Resurrected users were churned users who returned to using your app.

Retention is another core KPI that platforms use to measure how well they are at keeping their users.

The first step to calculating retention is getting each of the months in which each user is active.

In [ ]:
%%sql

-- Get the months in which each user is active
SELECT DISTINCT
  DATE_TRUNC('month', session_date) :: DATE AS month_active,
  user_id
FROM user_sessions
ORDER BY month_active ASC, user_id ASC;


 * postgresql+psycopg2://@/postgres


,month_active,user_id
0,2020-01-01,0
1,2020-01-01,1
2,2020-01-01,2
3,2020-01-01,3
4,2020-01-01,4
...,...,...
8685,2020-05-01,2056
8686,2020-05-01,2057
8687,2020-05-01,2058
8688,2020-05-01,2059


## **Self-joins**

Now that you have the months in which each user is active, how do you calculate retention?

![Left joins](https://user-images.githubusercontent.com/48436758/83518570-e4ff8c00-a4da-11ea-8a5a-25ea46df2bcc.png)

If you left-join this table on itself on the same user ID and having a one-month difference in users, you'll see whether a user is still active in the next month or not. If the user isn't active, then the user is churned. The count of non-`NULL`s is the count of retained users.

```sql
...
FROM ... AS prev
LEFT JOIN ... AS curr
  ON prev.user_id = curr.user_id
 AND prev.month = (curr.month - INTERVAL '1 MONTH')
...
```



In [ ]:
%%sql

-- Get whether each user churned in a given month
WITH act_months AS (
    SELECT DISTINCT
      DATE_TRUNC('month', session_date) :: DATE AS month_active,
      user_id
    FROM user_sessions)

SELECT 
    prev.user_id,
    prev.month_active,
    curr.month_active IS NULL AS churned_next_month
FROM act_months AS prev
LEFT JOIN act_months AS curr
  ON prev.user_id = curr.user_id
  AND prev.month_active = (curr.month_active - INTERVAL '1 MONTH')
WHERE curr.month_active IS NULL
ORDER BY prev.month_active ASC, prev.user_id ASC;

 * postgresql+psycopg2://@/postgres


,user_id,month_active,churned_next_month
0,34,2020-01-01,True
1,110,2020-01-01,True
2,147,2020-01-01,True
3,187,2020-01-01,True
4,216,2020-01-01,True
...,...,...,...
2188,2056,2020-05-01,True
2189,2057,2020-05-01,True
2190,2058,2020-05-01,True
2191,2059,2020-05-01,True


Store the results in a CTE and count the number of `FALSE` in the `churned_next_month` to get the retention rate.

In [ ]:
%%sql

-- Calculate the retention rate
WITH act_months AS (
    SELECT DISTINCT
      DATE_TRUNC('month', session_date) :: DATE AS month_active,
      user_id
    FROM user_sessions),

    churned AS (
SELECT 
    prev.user_id,
    prev.month_active,
    curr.month_active IS NULL AS churned_next_month
FROM act_months AS prev
LEFT JOIN act_months AS curr
  ON prev.user_id = curr.user_id
  AND prev.month_active = (curr.month_active - INTERVAL '1 MONTH')
    )

SELECT 
  month_active,
  COUNT(DISTINCT user_id) AS active_users,
  SUM( CASE WHEN churned_next_month THEN 0 ELSE 1 END
  ) AS retained_users,
  ROUND(
       SUM(
      CASE WHEN churned_next_month THEN 0 ELSE 1 END) :: NUMERIC /
        COUNT(DISTINCT user_id),2  ) AS retention_rate
FROM churned
GROUP BY month_active
ORDER BY month_active ASC;

 * postgresql+psycopg2://@/postgres


,month_active,active_users,retained_users,retention_rate
0,2020-01-01,1472,1455,0.99
1,2020-02-01,1621,1559,0.96
2,2020-03-01,1723,1667,0.97
3,2020-04-01,1879,1816,0.97
4,2020-05-01,1995,0,0.00


## **Average age of churners**

Now that you have the retention status of each user, you can see whether there are any trends in churns, such as older people churning more.


In [ ]:
%%sql

-- Get the average age of churners versus retained users in April
WITH act_months AS (
    SELECT DISTINCT
      DATE_TRUNC('month', session_date) :: DATE AS month_active,
      user_id
    FROM user_sessions),

    churned AS (
SELECT 
    prev.user_id,
    prev.month_active,
    curr.month_active IS NULL AS churned_next_month
FROM act_months AS prev
LEFT JOIN act_months AS curr
  ON prev.user_id = curr.user_id
  AND prev.month_active = (curr.month_active - INTERVAL '1 MONTH')
    )

    SELECT  
        churned_next_month,
        ROUND(AVG(age),2) AS avg_age
    FROM churned
    JOIN user_data ON churned.user_id = user_data.user_id
    WHERE month_active = '2020-04-01'
    GROUP BY churned_next_month

 * postgresql+psycopg2://@/postgres


,churned_next_month,avg_age
0,False,41.52
1,True,40.30


## **Q&A**